In [1]:
import pandas as pd
import os

In [2]:
BASE_DIR = os.getcwd()
BASE_DIR = os.path.dirname(BASE_DIR)  # sobe 1 nível
# BASE_DIR = os.path.dirname(os.path.dirname(BASE_DIR))  # sobe 2 níveis
# monta caminho relativo para o CSV
csv_path = os.path.join(BASE_DIR, "dados", "microdados_censo_escolar_2024", "dados", "microdados_ed_basica_2024.csv")
df = pd.read_csv(csv_path, encoding="latin1", delimiter=";", low_memory=False)
print(df.shape)



(215545, 426)


In [3]:
# FILTRO DE LINHAS

# 1. Escolas Privadas
df = df[df["TP_DEPENDENCIA"] == 4]
print(df.shape)

# 2. Em atividade
df = df[df["TP_SITUACAO_FUNCIONAMENTO"] == 1]
print(df.shape)

# 3. Excluindo Creches
cond_creche_pura = (
    (df["IN_INF_CRE"] == 1) &
    (df["IN_INF_PRE"] == 0) &
    (df["IN_FUND"] == 0) &
    (df["IN_MED"] == 0)
)
df = df[~cond_creche_pura]
print(df.shape)

# 4. Exlcuindo APAEs
df = df[~df["NO_ENTIDADE"].str.contains("APAE", case=False, na=False)]
print(df.shape)

# 5. Excluindo Sistema S: SENAI, SENAC, SESI
sistema_s = ["SESI", "SENAI", "SENAC", "SENAR", "SESC"]
df = df[~df["NO_ENTIDADE"].str.contains("|".join(sistema_s), case=False, na=False)]
print(df.shape)

# 6. Excluir escolas sem matrícula
# Soma das matrículas
df["QT_MAT_TOTAL"] = df[["QT_MAT_INF", "QT_MAT_FUND", "QT_MAT_MED"]].sum(axis=1, skipna=True)
cond_sem_matricula = (
    df[["QT_MAT_INF", "QT_MAT_FUND", "QT_MAT_MED"]].isna().all(axis=1)
) | (df["QT_MAT_TOTAL"] == 0)
df = df[~cond_sem_matricula]
print(df.shape)

# 7. Remove Ensino Tecnico EJA
df = df[df["IN_EJA"] != 1]
df = df[df["IN_PROF"] != 1]
print(df.shape)

(52568, 426)
(43218, 426)
(38289, 426)
(37517, 426)
(36270, 426)
(33730, 427)
(32306, 427)


In [4]:
# Add coluna de Cálculo de Receita Estimada

# Garantir que colunas são numéricas
cols = ["QT_MAT_INF", "QT_MAT_FUND_AI", "QT_MAT_FUND_AF", "QT_MAT_MED_PROP"]
for c in cols:
    df[c] = pd.to_numeric(df[c], errors="coerce")  # converte para número, NaN se não conseguir

# Valores médios por matrícula
valores = {
    "EI": 562.51,
    "EFAI": 1041.11,
    "EFAF": 1742.36,
    "EM": 2058.12
}

# Criar colunas de receita por segmento
df["REC_INF"] = df["QT_MAT_INF"] * valores["EI"]
df["REC_FUNDAI"] = df["QT_MAT_FUND_AI"] * valores["EFAI"]
df["REC_FUNDAF"] = df["QT_MAT_FUND_AF"] * valores["EFAF"]
df["REC_EM"] = df["QT_MAT_MED_PROP"] * valores["EM"]

# Receita total por escola
df["REC_TOTAL"] = (
    df["REC_INF"] +
    df["REC_FUNDAI"] +
    df["REC_FUNDAF"] +
    df["REC_EM"]
)

save_path = os.path.join(BASE_DIR, "dados", "escolas_filtradas.csv")
df.to_csv(save_path, sep=";", index=False, encoding="utf-8-sig", float_format="%.2f")

In [ ]:
# Adiconar colunas de coordenadas geográficas de cada escola
df_coordenadas = pd.read_csv('coordenadas.csv')

# 3. Fazer o merge
df = pd.merge(
    df,
    df_coordenadas,
    left_on='CO_CEP',
    right_on='CO_CEP',
    how='left'
)

# Salvar o novo dataset em um novo CSV
df.to_csv(save_path, sep=";", index=False, encoding="utf-8-sig", float_format="%.2f")


Merge concluído!
Formato do novo dataset (linhas, colunas): (32306, 434)

Visualização do dataset final:
   NU_ANO_CENSO NO_REGIAO  CO_REGIAO     NO_UF SG_UF  CO_UF NO_MUNICIPIO  \
0          2024     Norte          1  Rondônia    RO     11    Ariquemes   
1          2024     Norte          1  Rondônia    RO     11    Ariquemes   
2          2024     Norte          1  Rondônia    RO     11    Ariquemes   
3          2024     Norte          1  Rondônia    RO     11    Ariquemes   
4          2024     Norte          1  Rondônia    RO     11    Ariquemes   

   CO_MUNICIPIO NO_REGIAO_GEOG_INTERM  CO_REGIAO_GEOG_INTERM  ...  \
0       1100023           Porto Velho                   1101  ...   
1       1100023           Porto Velho                   1101  ...   
2       1100023           Porto Velho                   1101  ...   
3       1100023           Porto Velho                   1101  ...   
4       1100023           Porto Velho                   1101  ...   

  QT_TUR_FUND_AF_INT  Q